# Collecting the data

In [0]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar

# Processing the data

In [0]:
!rm -r data

In [30]:
!ls

datalab  imdb_crop  imdb_crop.tar


In [0]:
 !tar -xvf imdb_crop.tar

In [32]:
!ls imdb_crop

00  06	12  18	24  30	36  42	48  54	60  66	72  78	84  90	96
01  07	13  19	25  31	37  43	49  55	61  67	73  79	85  91	97
02  08	14  20	26  32	38  44	50  56	62  68	74  80	86  92	98
03  09	15  21	27  33	39  45	51  57	63  69	75  81	87  93	99
04  10	16  22	28  34	40  46	52  58	64  70	76  82	88  94	imdb.mat
05  11	17  23	29  35	41  47	53  59	65  71	77  83	89  95


In [0]:
import scipy.io
mat = scipy.io.loadmat('imdb_crop/imdb.mat')

In [34]:
mat

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Jan 17 11:30:27 2016',
 '__version__': '1.0',
 'imdb': array([[(array([[693726, 693726, 693726, ..., 726831, 726831, 726831]], dtype=int32), array([[1968, 1970, 1968, ..., 2011, 2011, 2011]], dtype=uint16), array([[array(['01/nm0000001_rm124825600_1899-5-10_1968.jpg'], dtype='<U43'),
         array(['01/nm0000001_rm3343756032_1899-5-10_1970.jpg'], dtype='<U44'),
         array(['01/nm0000001_rm577153792_1899-5-10_1968.jpg'], dtype='<U43'),
         ...,
         array(['08/nm3994408_rm926592512_1989-12-29_2011.jpg'], dtype='<U44'),
         array(['08/nm3994408_rm943369728_1989-12-29_2011.jpg'], dtype='<U44'),
         array(['08/nm3994408_rm976924160_1989-12-29_2011.jpg'], dtype='<U44')]],
       dtype=object), array([[1., 1., 1., ..., 0., 0., 0.]]), array([[array(['Fred Astaire'], dtype='<U12'),
         array(['Fred Astaire'], dtype='<U12'),
         array(['Fred Astaire'], dtype='<U12'), ..

In [0]:
mat['imdb'].shape

(1, 1)

In [35]:
dob = mat['imdb'][0, 0][0]
print(dob.shape)
dob

(1, 460723)


array([[693726, 693726, 693726, ..., 726831, 726831, 726831]], dtype=int32)

In [0]:
photo_taken = mat['imdb'][0, 0][1]
full_path = mat['imdb'][0, 0][2]
gender = mat['imdb'][0, 0][3]

### Calculating age

In [37]:
from datetime import datetime, timedelta
import numpy as np


train_len = 430000
test_len = dob.shape[1] - train_len

age = np.zeros_like(dob)
false_age = 0

for i in range(train_len):
    matlab_datenum = float(dob[0][i])
    try:
      python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)
      age[0][i] = np.abs(python_datetime.year - photo_taken[0][i])
    except:
      false_age += 1
      age[0][i] = 20

print(false_age)

111


### Create 100 directory for 100 age

In [0]:
!rm -r data

In [0]:
import os
import shutil

for i in range(1, 100):
    directory = 'data/train/'+str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    directory = 'data/test/'+str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [0]:
import random
n = dob.shape[1]

index = random.sample(range(0, n), n)

for i in range(train_len):
    src = full_path[0][index[i]][0]
    des = src.split('/')
    
    directory = 'data/train/'+str(age[0][index[i]])     
    src = "imdb_crop/"+str(des[0]+"/"+des[1])
    des =  directory+'/'+des[1]
    
    if age[0][index[i]]<100 and age[0][index[i]] != 0:
      shutil.copy(src, des)


for i in range(train_len, train_len + test_len):
    src = full_path[0][index[i]][0]
    des = src.split('/')
    
    directory = 'data/test/'+str(age[0][index[i]])      
    src = "imdb_crop/"+str(des[0]+"/"+des[1])
    des =  directory+'/'+des[1]
    
    if age[0][index[i]]<100 and age[0][index[i]] != 0:
      shutil.copy(src, des)
    

In [47]:
!ls data/train

1   14	19  23	28  32	37  41	46  50	55  6	64  69	73  78	82  87	91  96
10  15	2   24	29  33	38  42	47  51	56  60	65  7	74  79	83  88	92  97
11  16	20  25	3   34	39  43	48  52	57  61	66  70	75  8	84  89	93  98
12  17	21  26	30  35	4   44	49  53	58  62	67  71	76  80	85  9	94  99
13  18	22  27	31  36	40  45	5   54	59  63	68  72	77  81	86  90	95


# Building the model
This is a very big dataset with a lot of class and the accuracy of classification task is not expected to be very high. But still it is the first option to check. 

Here I am directly approaching with pretrained model

## Loading dataset

In [0]:
import os, shutil
train_dir = 'data/train'
test_dir = 'data/test'

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [50]:
from keras import models, layers, losses, optimizers, metrics
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=32
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=32
)

Found 401235 images belonging to 99 classes.
Found 28684 images belonging to 99 classes.


# Try using pretrined model

In [51]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))


58892288/58889256 [==============================] - 1s 0us/step


In [52]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [53]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(99, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dense_2 (Dense)              (None, 99)                50787     
Total params: 18,960,291
Trainable params: 18,960,291
Non-trainable params: 0
_________________________________________________________________


In [56]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=12600, 
                              epochs=10, 
                              validation_data=test_generator, 
                              validation_steps=900)

This is the number of trainable weights before freezing the conv base: 4
Epoch 1/10
  533/12600 [>.............................] - ETA: 37:53 - loss: 3.9338 - categorical_accuracy: 0.0400 1502/12600 [==>...........................] - ETA: 34:45 - loss: 3.9186 - categorical_accuracy: 0.0414 2469/12600 [====>.........................] - ETA: 31:40 - loss: 3.9142 - categorical_accuracy: 0.0407 3438/12600 [=======>......................] - ETA: 28:38 - loss: 3.9101 - categorical_accuracy: 0.0411 4407/12600 [=========>....................] - ETA: 25:35 - loss: 3.9086 - categorical_accuracy: 0.0412 5374/12600 [===========>..................] - ETA: 22:34 - loss: 3.9067 - categorical_accuracy: 0.0416 6343/12600 [==============>...............] - ETA: 19:32 - loss: 3.9042 - categorical_accuracy: 0.0418 7310/12600 [================>.............] - ETA: 16:31 - loss: 3.9023 - categorical_accuracy: 0.0419 8279/12600 [==================>...........] - ETA: 13:29 - loss: 3.9012 - categorical_accur

In [0]:
model.save('model.h5')

from google.colab import files
files.download('model.h5')

In [0]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()